In [9]:
import pathlib as pl
from PIL import Image
import datasets
from datasets import Dataset
from datasets import Image as DSImage
import numpy as np
import cv2
# files = list((pl.Path(__file__).parent / "inference_inputs").glob("*"))
files = list((pl.Path(".").parent / "inference_inputs").glob("*"))
print(files)
# exit()


[PosixPath('inference_inputs/018_0005855_0005880.tiff')]


In [6]:

def window_with_remainder(length, overlap, input_size):
    testarray = np.arange(0, input_size)
    return np.vstack(
        (
            testarray[0:length],
            np.lib.stride_tricks.sliding_window_view(
                testarray[len(testarray) % length :], length
            )[::overlap],
        )
    )[:, [0, -1]] + [0, 1]

# Converts an image from cv2 to PIL
def convert_from_cv2_to_image(img: np.ndarray) -> Image:
    # return Image.fromarray(img)
    return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


# Converts an image from PIL to cv2
def convert_from_image_to_cv2(img: Image) -> np.ndarray:
    # return np.asarray(img)
    return cv2.cvtColor(src=np.array(img, dtype=np.uint8), code=cv2.COLOR_RGB2BGR)


# Splits an longer image into a series of smaller images with a given overlap
# to reduce memory costs it takes in the length, input size and returns a list of valid indices
def window_with_remainder(length, overlap, input_size):
    testarray = np.arange(0, input_size)
    return np.vstack(
        (
            testarray[0:length],
            np.lib.stride_tricks.sliding_window_view(
                testarray[len(testarray) % length :], length
            )[::overlap],
        )
    )[:, [0, -1]] + [0, 1]


# does canny edge detection on an image
def canny_image(image: Image):
    # Read the image

    # Convert the image to grayscale
    gray = cv2.cvtColor(convert_from_image_to_cv2(image), cv2.COLOR_BGR2GRAY)

    mean_power = np.mean(gray) / 255
    print(f"Mean power of the grayscale image: {mean_power}")
    std = np.std(gray) / 255
    print(f"Variance of the grayscale image: {std}")

    t1 = ((mean_power - (2 * std)) * 255, (mean_power - (1 * std)) * 255)
    t2 = ((mean_power + (1 * std)) * 255, (mean_power + (2 * std)) * 255)
    t3 = ((mean_power - (1 * std)) * 255, (mean_power + (1 * std)) * 255)

    # Perform Canny edge detection on the grayscale image with two different thresholds
    edges1 = cv2.Canny(gray, threshold1=t1[0], threshold2=t1[1])
    edges2 = cv2.Canny(
        cv2.GaussianBlur(gray, (11, 11), std * 4, std / 2),
        threshold1=t2[0],
        threshold2=t2[1],
    )
    edges3 = cv2.Canny(gray, threshold1=t3[0], threshold2=t3[1])

    kernel = np.ones((10, 10), np.uint8)
    edges3 = cv2.dilate(edges3, kernel, iterations=1)

    combined_image = np.zeros((gray.shape[0], gray.shape[1], 3), dtype=np.uint8)
    edges1 = cv2.Canny(gray, threshold1=t1[0], threshold2=t1[1])
    edges2 = cv2.Canny(
        cv2.GaussianBlur(gray, (11, 11), std * 4, std / 2),
        threshold1=t2[0],
        threshold2=t2[1],
    )
    edges3 = cv2.Canny(gray, threshold1=t3[0], threshold2=t3[1])

    kernel = np.ones((10, 10), np.uint8)
    edges3 = cv2.dilate(edges3, kernel, iterations=1)

    combined_image = np.zeros((gray.shape[0], gray.shape[1], 3), dtype=np.uint8)

    combined_image[:, :, 0] = gray

    combined_image[:, :, 1] = edges1 + edges3 * 0.3
    combined_image[:, :, 0] = gray

    combined_image[:, :, 1] = edges1 + edges3 * 0.3

    # Set the third color channel as edges2
    combined_image[:, :, 2] = edges2 + edges3 * 0.3
    combined_image[:, :, 2] = edges2 + edges3 * 0.3

    return convert_from_cv2_to_image(combined_image)

In [16]:
i.stem

'018_0005855_0005880'

In [20]:

image_paths = []
for i in files:
    img = Image.open(i)
    img = img.crop((0, 430, img.size[0], 1790))
    img = img.convert("L")
    cropping_coordinates = window_with_remainder(512, 128, img.size[0])
    file_path = pl.Path("./inference_raw_imgs") / i.stem
    file_path.mkdir(exist_ok=True)
    for count, (start, end) in enumerate(cropping_coordinates):
        save_path = file_path / f"{count}-{start}-{end}.png"
        resized = canny_image(img.crop((start, 0, end, img.size[1])).resize((512, 512)))
        resized.save(save_path)    
        print(f"finished {save_path}")
        image_paths.append(str(save_path.resolve()))
    print("\n--\n")



/home/users/aashrayc/.local/lib/python3.12/site-packages/PIL/Image.py:3368: DecompressionBombWarning: Image size (94704000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Mean power of the grayscale image: 0.3016998291015625
Variance of the grayscale image: 0.10794610837099866
finished inference_raw_imgs/018_0005855_0005880/0-0-512.png
Mean power of the grayscale image: 0.30229882632984834
Variance of the grayscale image: 0.10774013860287023
finished inference_raw_imgs/018_0005855_0005880/1-36-548.png
Mean power of the grayscale image: 0.3026700936111749
Variance of the grayscale image: 0.10712157985376101
finished inference_raw_imgs/018_0005855_0005880/2-164-676.png
Mean power of the grayscale image: 0.3025839712105545
Variance of the grayscale image: 0.10659776565898407
finished inference_raw_imgs/018_0005855_0005880/3-292-804.png
Mean power of the grayscale image: 0.30258976057464
Variance of the grayscale image: 0.10629906510728918
finished inference_raw_imgs/018_0005855_0005880/4-420-932.png
Mean power of the grayscale image: 0.30248034608130364
Variance of the grayscale image: 0.10598252586674554
finished inference_raw_imgs/018_0005855_0005880/5-5

In [21]:
dataset = Dataset.from_dict({"image": sorted(image_paths)})
dataset = dataset.cast_column("image", DSImage())
# prints dataset information
print(dataset)
print(dataset.info)
print(dataset.features)

dataset.save_to_disk("./inference_dataset/inference_dataset.hf")



Dataset({
    features: ['image'],
    num_rows: 306
})
DatasetInfo(description='', citation='', homepage='', license='', features={'image': Image(mode=None, decode=True, id=None)}, post_processed=None, supervised_keys=None, builder_name=None, dataset_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)
{'image': Image(mode=None, decode=True, id=None)}


Saving the dataset (1/1 shards): 100%|██████████| 306/306 [00:01<00:00, 280.32 examples/s]
